This code pulls variables from multiple clean and processed csvs to create the dataframes used in the behavioral analyses. It creates one dataframe per main analysis: one for the ordinal model of the first clips of each observation, one for the survival analysis of clips throughout the observations, and one comparing the behavior of the groups before and after the drone exchanges.

In [1]:
# import packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

os.getcwd()

'/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement'

In [10]:
# read in data
meta = pd.read_csv('clean-data/metadata.csv')
first_clips = pd.read_csv('clean-data/first_clips_clean.csv')
whole_ob = pd.read_csv('clean-data/whole_observation_clean.csv')
exch = pd.read_csv('clean-data/exchanges_clean.csv')
flight = pd.read_csv('processed-data/drone_variables_flight.csv')
clip = pd.read_csv('processed-data/drone_variables_clip2.csv')
env = pd.read_csv('processed-data/env_variables_clip2.csv')
ground = pd.read_csv('clean-data/ground_conditions.csv')

In [11]:
# Create dataframe for analysis of first clips
df1 = pd.merge(meta, first_clips, on = 'flight', how = 'outer')
#df1 = pd.merge(df1, flight, on = 'flight', how = 'outer')
#df1 = pd.merge(df1, clip, on = 'flight', how = 'outer')
#df1 = pd.merge(df1, env, on = 'flight', how = 'outer')
#df1 = pd.merge(df1, ground, on = 'flight', how = 'outer')
#df1.replace('NAN', np.nan, inplace = True)

In [12]:
df1

,observation_x,site_x,date,map_area,flight,observation_y,site_y,species,group_size_spp,group_size_total,young_present,note,exclude,group_behavior
0,ob001,opc,11/2/17,NAN,001-01,ob001,opc,gg,5.0,21.0,False,NaN,NaN,2.0
1,ob001,opc,11/2/17,NAN,001-01,ob001,opc,tg,16.0,21.0,True,NaN,NaN,1.0
2,ob002,opc,11/2/17,NAN,002-01,ob002,opc,buf,14.0,14.0,False,NaN,NaN,1.0
3,ob003,opc,11/2/17,NAN,003-01,ob003,opc,tg,5.0,26.0,True,NaN,NaN,NaN
4,ob003,opc,11/2/17,NAN,003-01,ob003,opc,pz,21.0,26.0,True,NaN,NaN,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,ob115,mpala,4/19/18,airstrip_ne_merge,115-01,ob115,mpala,gz,1.0,30.0,False,NaN,NaN,0.0
261,ob115,mpala,4/19/18,airstrip_ne_merge,115-01,ob115,mpala,pz,19.0,30.0,True,NaN,NaN,0.0
262,ob115,mpala,4/19/18,airstrip_ne_merge,115-01,ob115,mpala,oryx,10.0,30.0,True,NaN,NaN,0.0
263,ob115,mpala,4/19/18,airstrip_ne_merge,115-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Gather all columns in one dataframe, matching up by flight and clip_type
new_df = pd.merge(meta, beh, on = 'flight', how = 'outer')
new_df = pd.merge(new_df, flight, on = 'flight', how = 'outer')
new_df = pd.merge(new_df, clip, on = ['flight'], how = 'outer')
new_df = pd.merge(new_df, env, on = ['flight'], how = 'outer')
new_df = pd.merge(new_df, ground, on = 'flight', how = 'outer')
new_df.replace('NAN', np.nan, inplace = True)

In [13]:
# add in flight type (initial or replacement)
new_df['flight_type'] = ['initial' if '-01' in i else 'replacement' for i in new_df['flight']]

In [15]:
# determine whether group is downwind or not
new_df['wind_direction'] = pd.to_numeric(new_df['wind_direction'])
new_df['angle'] = abs(new_df['wind_direction'] - new_df['launch-group_bear'])
new_df['wind_angle'] = [i if i < 180 else 360-i for i in new_df['angle']]
new_df['group_downwind'] = [True if i <45 else False for i in new_df['wind_angle']]

In [17]:
cols_to_keep = ['observation', 'site', 'date', 'flight', 'flight_type', 'species',
       'clip_type', 'group_size_spp', 'group_size_total', 'young_present', 'group_response',
       'launch_alt', 'group_lat', 'group_lon', 'obs_ascent', 'launch_dist', 'approach_speed_mean', 'approach_speed_max',
       'approach_duration_minutes', 'approach_tortuousity', 'exchange_duration', 'takeoff_time', 'landing_time', 'clip_start_ascent', 'prox_move_v',
       'prox_move_h', 'bushiness_100m', 'max_windspeed', 'mean_windspeed', 'temperature_c',
       'wind_speed_ms', 'cloud_cover_%', 'humidity_%', 'group_downwind', 'clip_start_ascent_adj']
new_df.drop(new_df.columns.difference(cols_to_keep), axis = 1, inplace = True)
# new_df.rename(columns={'launch_alt_x':'launch_alt'}, inplace=True)
new_df.to_csv('processed-data/master_df.csv', index = False)